In [365]:
import duckdb
import pandas as pd
from fredapi import Fred
import certifi
import os
from datetime import datetime, timedelta, date
from config import FRED_API_KEY
os.environ['SSL_CERT_FILE'] = certifi.where() #API Key security requirements
fred = Fred(api_key=FRED_API_KEY)
today_date = date.today().strftime('%Y-%m-%d') #Fetch Date

In [366]:
# Master List of FRED Indexes:
series_ids = [
    "DCOILWTICO", "GASREGW", "DPROPANEMBTX", "CPIAUCSL", "PPIACO", "PCE", "CPILFESL", "UMCSENT", "PCEPILFE", "MICH",
    "PAYEMS", "UNRATE", "CES0500000003", "CIVPART", "JTSJOL", "ICSA", "ADPWNUSNERSA", "U6RATE", "AWHNONAG", "FEDFUNDS", 
    "DFEDTAR", "IOER", "DISCOUNT", "DGS10", "SOFR", "PRIME", "TB3MS", "CPFF", "CSUSHPINSA", "HSN1F", "PERMIT", "HOUST", 
    "USSTHPI", "EXHOSLUSM495S", "MORTGAGE30US", "RRVRUSQ156N", "RHORUSQ156N", "HOSINVUSM495N"
]

# Energy Indexes:
energy_indexes =  ['DCOILWTICO', 'GASREGW', 'DPROPANEMBTX']
# CPI/PPI/Consumer Sentiment Indexes:
cpi_ppi_consumer_sentiment_indexes =  ['CPIAUCSL', 'PPIACO', 'PCE', 'CPILFESL', 'UMCSENT', 'PCEPILFE', 'MICH']
# Employment Indexes:
employment_indexes =  ['PAYEMS', 'UNRATE', 'CES0500000003', 'CIVPART', 'JTSJOL', 'ICSA', 'ADPWNUSNERSA', 'U6RATE', 'AWHNONAG']
# Federal Funds Indexes:
federal_funds_indexes =  ['FEDFUNDS', 'DFEDTAR', 'IOER', 'DISCOUNT', 'DGS10', 'SOFR', 'PRIME', 'TB3MS', 'CPFF']
# Housing Indexes:
housing_indexes =  ['CSUSHPINSA', 'HSN1F', 'PERMIT', 'HOUST', 'USSTHPI', 'EXHOSLUSM495S', 'MORTGAGE30US', 'RRVRUSQ156N', 'RHORUSQ156N', 'HOSINVUSM495N']

### Fetch Data from the Economic Research Federal Reserve Bank of St. Louis

In [367]:
def fetch_series_data(series_id):
    """
    Fetch series data for a given series ID between start_date and end_date.
    
    Parameters:
    - series_id (str): The FRED series ID.
    
    Returns:
    - pd.Series: The time series data.
    """
    try:
        series_data = fred.get_series(series_id)
        return series_data
    except Exception as e:
        print(f"Error fetching series data for {series_id}: {e}")
        return pd.Series([])

def fetch_metadata(series_id):
    """
    Fetch metadata for a given series ID.
    
    Parameters:
    - series_id (str): The FRED series ID.
    
    Returns:
    - dict: The metadata for the series.
    """
    try:
        metadata = fred.get_series_info(series_id)
        return metadata
    except Exception as e:
        print(f"Error fetching metadata for {series_id}: {e}")
        return {}

def construct_dataframe(series_data, metadata, series_id, fetch_date):
    """
    Construct a DataFrame from series data and metadata.
    
    Parameters:
    - series_data (pd.Series): The time series data.
    - metadata (dict): The metadata for the series.
    - series_id (str): The FRED series ID.
    - fetch_date (str): The date when the data was fetched.
    
    Returns:
    - pd.DataFrame: The constructed DataFrame with data and metadata.
    """
    try:
        df = pd.DataFrame(series_data, columns=['series_index']).reset_index().rename(columns={'index': 'date'})
        df['name'] = metadata.get('title', '')
        df['seriesid'] = series_id
        df['source'] = f'https://fred.stlouisfed.org/series/{series_id}'
        df['units'] = metadata.get('units', '')
        df['seasonal_adjustment'] = metadata.get('seasonal_adjustment', '')
        df['frequency'] = metadata.get('frequency', '')
        df['fred_last_updated_date'] = metadata.get('last_updated', '')
        df['fetch_date'] = fetch_date
        return df
    except Exception as e:
        print(f"Error constructing dataframe for {series_id}: {e}")
        return pd.DataFrame()

def fetch_fred_data(series_ids):
    """
    Fetch data and metadata for a list of FRED series IDs.
    
    Parameters:
    - series_ids (list of str): The FRED series IDs.
    
    Returns:
    - list of pd.DataFrame: A list of DataFrames with data and metadata for each series.
    """
    today_date = date.today().strftime('%Y-%m-%d')
    dataframes = []
    
    for series_id in series_ids:
        series_data = fetch_series_data(series_id)
        if series_data.empty:
            continue
        metadata = fetch_metadata(series_id)
        df = construct_dataframe(series_data, metadata, series_id, today_date)
        dataframes.append(df)
    
    return dataframes


In [368]:
# Fetch data
dataframes = fetch_fred_data(series_ids)

### Connect to DuckDB and Iterate and Place Tables in the DuckDB database

In [369]:
# Creates empty database and shows existing tables
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name


In [370]:
conn = duckdb.connect('data/fred.db')
for series_id, dataframe in zip(series_ids, dataframes):
    # Convert series ID to lowercase for the table name
    table_name = series_id.lower()
    
    # Register the DataFrame as a virtual table
    conn.register(table_name + "_temp", dataframe)
    
    # Create or replace the physical table from the virtual table
    conn.execute(f'''
    CREATE OR REPLACE TABLE {table_name} AS 
    SELECT * FROM {table_name}_temp
    ''')

    # Optionally, unregister the virtual table to clean up
    conn.unregister(table_name + "_temp")
conn.close()

sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred.db') as con: #using with to automatically close the database
    display(con.sql(sql_query).df())

,name
0,adpwnusnersa
1,awhnonag
2,ces0500000003
3,civpart
4,cpff
5,cpiaucsl
6,cpilfesl
7,csushpinsa
8,dcoilwtico
9,dfedtar


### Create Bronze Layer of Medallion Architecture

In [371]:
def create_bronze_sql_files(series_ids, target_directory):
    """
    Create .sql files in the target directory for a list of series IDs.
    
    Parameters:
    - series_ids (list of str): The FRED series IDs.
    - target_directory (str): The target directory to store .sql files.
    """
    # Ensure the target directory exists, create if not
    bronze_directory = os.path.join(target_directory, "bronze")
    os.makedirs(bronze_directory, exist_ok=True)
    
    # SQL template
    sql_template = """SELECT 
date AS Date,
series_index AS Series,
name AS Name,
units AS Units,
seasonal_adjustment AS AdjSeas,
frequency AS Freq,
fred_last_updated_date AS LastUpdatedDate,
fetch_date AS FetchDate
FROM {}
"""
    
    # Iterate over series IDs and create SQL files
    for series_id in series_ids:
        file_name = f"bronze_{series_id.lower()}.sql"
        file_path = os.path.join(bronze_directory, file_name)
        
        # Format the SQL query for the current series ID
        sql_query = sql_template.format(series_id.lower())
        
        # Write the SQL query to a file
        with open(file_path, 'w') as sql_file:
            sql_file.write(sql_query)
            
        print(f"SQL file created: {file_path}")

In [372]:
target_directory = "/Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models"
create_bronze_sql_files(series_ids, target_directory)

SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_dcoilwtico.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_gasregw.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_dpropanembtx.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_cpiaucsl.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_ppiaco.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_pce.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_cpilfesl.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_umcsent.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economi

### Create Silver Layer of Medallion Architecture (Union All)

In [373]:
def create_silver_sql_files(index_categories, target_directory):
    """
    Create .sql files in the 'silver' subdirectory for unioning tables related to each category.

    Parameters:
    - index_categories (dict of list): A dictionary where each key is a category name and each value is a list of series IDs.
    - target_directory (str): The target directory to store .sql files.
    """
    # Ensure the target directory exists, create if not
    silver_directory = os.path.join(target_directory, "silver")
    os.makedirs(silver_directory, exist_ok=True)

    # Iterate over each category and create SQL files
    for category_name, series_ids in index_categories.items():
        file_name = f"silver_{category_name}_unionall.sql"
        file_path = os.path.join(silver_directory, file_name)
        
        # Start the SQL query for UNION ALL
        sql_queries = []
        for series_id in series_ids:
            table_name = series_id.lower()
            sql_query = f"SELECT Date, \
            \nSeries, \
            \nName, \
            \nUnits, \
            \nAdjSeas, \
            \nFreq, \
            \nLastUpdatedDate, \
            \nFetchDate FROM {{{{ref('bronze_%s')}}}}" % table_name
            sql_queries.append(sql_query)
        union_sql = " \nUNION ALL \n".join(sql_queries)
        
        # Write the SQL query to a file
        with open(file_path, 'w') as sql_file:
            sql_file.write(union_sql)
            
        print(f"SQL file created: {file_path}")

In [374]:
index_categories = {
    'energy': energy_indexes,
    'cpi_ppi_sentiment': cpi_ppi_consumer_sentiment_indexes,
    'employment': employment_indexes,
    'federal_funds': federal_funds_indexes,
    'housing': housing_indexes
}

target_directory = "/Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models"
create_silver_sql_files(index_categories, target_directory)

SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/silver_energy_unionall.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/silver_cpi_ppi_sentiment_unionall.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/silver_employment_unionall.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/silver_federal_funds_unionall.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/silver/silver_housing_unionall.sql


### Create Silver Layer of Medallion Architecture (AdjSeas - Boolean)

In [375]:
# f"SELECT Date, \             
# Series, \            
# Name, \            
# Units, \            
# CASE 
#     WHEN AdjSeas = 'Not Seasonaly Adjusted' THEN FALSE
#     WHEN AdjSeas = 'Seasonaly Adjusted' THEN TRUE
#     ELSE NULL END AS AdjSeas, \            
# Freq, \            
# LastUpdatedDate, \            
# FetchDate FROM {{{{ref('silver_%s')}}}}" % table_name

### Create Gold Layer of Medallion Architecture